In [6]:
!pip install w3lib

## Training

In [ ]:
import os
import argparse
import time
import json
import torch
import random
import torch.nn as nn
import numpy as np
from utils import initNetParams, train_step, val_step
from dataset import EurDataset, collate_data
from models.transformer import Model
from torch.utils.data import DataLoader
from tqdm import tqdm
import pandas as pd
from models.autoencoders import AE


In [ ]:
parser = argparse.ArgumentParser()
#parser.add_argument('--data-dir', default='data/train_data.pkl', type=str)
parser.add_argument('--vocab-file', default='data/vocab.json', type=str)
parser.add_argument('--checkpoint-path', default='checkpoints/deepsc-Rayleigh', type=str)
parser.add_argument('--channel', default='Rayleigh', type=str, help = 'Please choose AWGN, Rayleigh, and Rician')
parser.add_argument('--MAX-LENGTH', default=30, type=int)
parser.add_argument('--MIN-LENGTH', default=4, type=int)
parser.add_argument('--d-model', default=128, type=int)
parser.add_argument('--dff', default=512, type=int)
parser.add_argument('--num-layers_deepsc', default=4, type=int)
parser.add_argument('--num-layers_ae', default=6, type=int)
parser.add_argument('--num-heads', default=8, type=int)
parser.add_argument('--batch-size', default=128, type=int)
parser.add_argument('--epochs', default=8, type=int)
parser.add_argument('--ae_cpath', default="data/checks/", type=str)
# parser.add_argument('--ae_cpath', default="/kaggle/working/checks/ae", type = str)

_StoreAction(option_strings=['--ae_cpath'], dest='ae_cpath', nargs=None, const=None, default='/kaggle/input/datasc/checks/checks', type=<class 'str'>, choices=None, required=False, help=None, metavar=None)

In [10]:
# # !pip install torchsummary
# from torchsummary import summary
# from autoencodersc.autoencodersc import AE
# vocab = json.load(open('/kaggle/input/datasc/vocab.json', 'rb'))
# token_to_idx = vocab['token_to_idx']
# num_vocab = len(token_to_idx)
# model1 = Model(4, num_vocab, num_vocab,
#                     num_vocab, num_vocab, 128, 8,
#                     512, 0.1).to(device)
# # mod = model1.encoder
# # ae = AE().to(device)
# # summary(mod, input_size=(26, num_vocab))
# print(model1)

train_eur= EurDataset('test')
print(len(train_eur.data))
  
# train_iterator = DataLoader(train_eur, batch_size=args.batch_size, num_workers=0,
#                             pin_memory=True, collate_fn=collate_data)
# pbar = tqdm(train_iterator)

25899


In [13]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def setup_seed(seed): 
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    np.random.seed(seed)
    random.seed(seed)
    torch.backends.cudnn.deterministic = True

def validate(epoch, args, net, type, ae=None):
    test_eur = EurDataset('test')
    test_iterator = DataLoader(test_eur, batch_size=args.batch_size, num_workers=0,
                                pin_memory=True, collate_fn=collate_data)
    net.eval()
    pbar = tqdm(test_iterator)
    total = 0
    with torch.no_grad():
        for sents in pbar:
            sents = sents.to(device)
            if type != 3: #else
                loss = val_step(net, sents, sents, pad_idx,
                                 criterion, type)
            if type == 3:
                loss = val_step(net, sents, sents, pad_idx,
                                 criterion, type, ae, args.ae_cpath)

            total += loss
            pbar.set_description(
                'Epoch: {}; Type: VAL; Loss: {:.5f}'.format(
                    epoch + 1, loss
                )
            )

    return total/len(test_iterator)


In [ ]:
def train(epoch, args, net, optimizer, type, ae=None):
    print("Train Model Type", type)
    train_eur= EurDataset('train')
  
    train_iterator = DataLoader(train_eur, batch_size=args.batch_size, num_workers=0,
                                pin_memory=True, collate_fn=collate_data)
    pbar = tqdm(train_iterator)

    # noise_std = np.random.uniform(SNR_to_noise(5), SNR_to_noise(10), size=(1))
    # enc_op_list = []
    
    for sents in pbar:
        sents = sents.to(device)

        if type != 3:
            loss, enc_op = train_step(net, sents, sents, pad_idx,
                                optimizer, criterion, type)
        if type == 3:
            loss, enc_op = train_step(net, sents, sents, pad_idx,
                            optimizer, criterion, type, ae, args.ae_cpath)
        pbar.set_description(
            'Epoch: {};  Type: Train; Loss: {:.5f}'.format(
                epoch + 1, loss
            )
        )
        # """Extract encoded tensors"""
        # enc_op_list.append(enc_op.cpu().detach().numpy())
    # print(len(enc_op_list))
    
    # print(len(enc_op_list[0]), len(enc_op_list[0][0]), len(enc_op_list[0][0][0]))
    # try:
    #     print(len(enc_op_list[0][0][0][0]))
    # except:
    #     print("Nope")
    # data = []
    # for batch in enc_op_list:
    #     for line in batch:
    #         data.append(line)
    # op1 = pd.DataFrame({"enc": data, "dec": data})
    # op1.to_csv("enc_op.csv", encoding='utf-8', index=False)


In [ ]:
%%time

if __name__ == '__main__':
    # setup_seed(10)
    args, unknown = parser.parse_known_args()
    
    # args.vocab_file = './' + args.vocab_file
    args.vocab_file = args.vocab_file

    """ preparing the dataset """
    vocab = json.load(open(args.vocab_file, 'rb'))
    token_to_idx = vocab['token_to_idx']
    num_vocab = len(token_to_idx)
    pad_idx = token_to_idx["<PAD>"]
    start_idx = token_to_idx["<START>"]
    end_idx = token_to_idx["<END>"]


    """ define optimizer and loss function """
    model1 = Model(args.num_layers_deepsc, num_vocab, num_vocab,
                        num_vocab, num_vocab, args.d_model, args.num_heads,
                        args.dff, 0.1).to(device)
    # for types 3, 4, enc blocks = 6
    model2 = Model(args.num_layers_ae, num_vocab, num_vocab,
                        num_vocab, num_vocab, args.d_model, args.num_heads,
                        args.dff, 0.1).to(device)
    

    """Define type and model"""

    type = 0 #type can be 0, 1, 3 is automaticlly model2
    deepsc = model1
    
    criterion = nn.CrossEntropyLoss(reduction = 'none')
    optimizer1 = torch.optim.Adam(deepsc.parameters(),
                                 lr=1e-4, betas=(0.9, 0.98), eps=1e-8, weight_decay = 5e-4)
    optimizer2 = torch.optim.Adam(deepsc.parameters(),
                                 lr=2e-4, betas=(0.9, 0.98), eps=1e-8, weight_decay = 5e-4)
    #opt = NoamOpt(args.d_model, 1, 4000, optimizer)
    initNetParams(deepsc)
    # for param_tensor in deepsc.state_dict():
    #     print(param_tensor, "\t", deepsc.state_dict()[param_tensor].size())
    pths = ['checkpoints/deepsc-1', 'checkpoints/deepsc-2']
    optimizers = {"1":optimizer1,"2":optimizer2}
    # args.checkpoint_path ='checkpoints/deepsc-Rayleigh1'
    # args.checkpoint_path ='checkpoints/deepsc-Rayleigh2'
    for p in pths:
        print(p, p[-1])
        args.checkpoint_path = p
        op = optimizers[p[-1]]

        for epoch in range(args.epochs):
            start = time.time()
            record_acc = 10
            
            """Check if type 3"""
            if type == 3:
                ae = AE().to(device)
                train(epoch, args, deepsc, op, type, ae) 
                avg_acc = validate(epoch, args, deepsc, type, ae)
            else:
                train(epoch, args, deepsc, op, type) 
                avg_acc = validate(epoch, args, deepsc, type)
            # print(avg_acc)
            if avg_acc < record_acc:
                if not os.path.exists(args.checkpoint_path):
                    os.makedirs(args.checkpoint_path)
                with open(args.checkpoint_path + '/checkpoint_{}.pkl'.format(str(epoch + 1).zfill(2)), 'wb') as f:
                    torch.save(deepsc.state_dict(), f)
                record_acc = avg_acc
        record_loss = []


## Performance

In [ ]:
import os
import json
import torch
import argparse
import numpy as np
from dataset import EurDataset, collate_data
from models.transformer import Model
from torch.utils.data import DataLoader
from utils import BleuScore, greedy_decode, SeqtoText
from tqdm import tqdm
from sklearn.preprocessing import normalize
# from bert4keras.backend import keras
# from bert4keras.models import build_bert_model
# from bert4keras.tokenizers import Tokenizer
from w3lib.html import remove_tags
from models.autoencoders import AE

In [ ]:
parser = argparse.ArgumentParser()
# parser.add_argument('--data-dir', default='europarl/train_data.pkl', type=str)
# parser.add_argument('--vocab-file', default='europarl/vocab.json', type=str)
parser.add_argument('--data-dir', default='data/train_data.pkl', type=str)
parser.add_argument('--vocab-file', default='data/vocab.json', type=str)
parser.add_argument('--checkpoint-path', default='checkpoints/deepsc', type=str)
# parser.add_argument('--channel', default='Rayleigh', type=str)
parser.add_argument('--MAX-LENGTH', default=30, type=int)
parser.add_argument('--MIN-LENGTH', default=4, type=int)
parser.add_argument('--d-model', default=128, type = int)
parser.add_argument('--dff', default=512, type=int)
parser.add_argument('--num-layers_deepsc', default=4, type=int)
parser.add_argument('--num-layers_ae', default=6, type=int)
parser.add_argument('--num-heads', default=8, type=int)
parser.add_argument('--batch-size', default=64, type=int)
parser.add_argument('--epochs', default=2, type = int)
parser.add_argument('--ae_cpath', default="data/checks", type=str)
# parser.add_argument('--ae_cpath', default="/kaggle/working/checks/ae", type = str)

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


In [ ]:
%%time
import numpy as np
def performance(args, net, lr, type, ae=None):
    # similarity = Similarity(args.bert_config_path, args.bert_checkpoint_path, args.bert_dict_path)
    bleu_score_1gram = BleuScore(1, 0, 0, 0)
    bleu_score_2gram = BleuScore(0, 1, 0, 0)
    bleu_score_3gram = BleuScore(0, 0, 1, 0)
    bleu_score_4gram = BleuScore(0, 0, 0, 1)
    bleu_score_avg = BleuScore(0.25, 0.25, 0.25, 0.25)    
    
    # gram_dict = {"one":bleu_score_1gram, "two":bleu_score_2gram,
    #             "three":bleu_score_3gram, "four":bleu_score_4gram, "avg":bleu_score_avg}

    # bleu_score_gram = gram_dict[gram]    
    
    test_eur = EurDataset('test')
    test_iterator = DataLoader(test_eur, batch_size=args.batch_size, num_workers=0,
                               pin_memory=True, collate_fn=collate_data)

    StoT = SeqtoText(token_to_idx, end_idx)
    scoreavg = []
    score1 = []
    score2 = []
    score3 = []
    score4 = []
    
    net.eval()
    
    print("Test Model Type", type)
    with torch.no_grad():
        for epoch in (range(args.epochs)):
            print("In Epoch {}".format(epoch))
            
            Tx_word = []
            Rx_word = []
            for i in range(1):
                word = []
                target_word = []
                print("Encoding-Decoding")
                for sents in tqdm(test_iterator):
    
                    sents = sents.to(device)
                    # src = batch.src.transpose(0, 1)[:1]
                    target = sents
                    if type != 3:
                        out = greedy_decode(net, sents, args.MAX_LENGTH, pad_idx,
                                                start_idx, type)
                    if type == 3:
                        out = greedy_decode(net, sents, args.MAX_LENGTH, pad_idx,
                                            start_idx, type, ae, args.ae_cpath)
                    # print("out####################", len(out), len(out[0]),len(out[1]))
    
                    sentences = out.cpu().numpy().tolist()
                    result_string = list(map(StoT.sequence_to_text, sentences))
                    word = word + result_string
                    # print("word###################", (result_string))
    
                    target_sent = target.cpu().numpy().tolist()
                    result_string = list(map(StoT.sequence_to_text, target_sent))
                    target_word = target_word + result_string
                    # print("w********************", (result_string))

                Tx_word.append(word)
                Rx_word.append(target_word)
                
            # op_to_csv = {"sent1" : Tx_word, "sent2" : Rx_word}
            # op = pd.DataFrame(op_to_csv)
            # print(op.head())
            # op.to_csv("op.csv", sep = "\t", encoding='utf-8')
            # print(len(Tx_word))    
            bleu_score = []
            bleu1 = []
            bleu2 = []
            bleu3 = []
            bleu4 = []

            # op_to_csv = {"sent1" : Tx_word, "sent2" : Rx_word}
            # op = pd.DataFrame(op_to_csv)
            # print(op.head(25))
            # op.to_csv("op.csv", encoding='utf-8')
            s1 = []
            s2 = []
            print("Scoring")
            for sent1, sent2 in tqdm(zip(Tx_word, Rx_word)):#cos no snr, len of txword is 1
                #therefore there is only 1 list to iter over, ie., 1 sent1, 2
                
                # print(sent1[1], sent2[1])
                # 1-gram
                s1.append(sent1)
                s2.append(sent2)
                a = time.time()
                bleu1.append(bleu_score_1gram.compute_blue_score(sent1, sent2)) # 7*num_sent
                bleu2.append(bleu_score_2gram.compute_blue_score(sent1, sent2))
                bleu3.append(bleu_score_3gram.compute_blue_score(sent1, sent2))
                bleu4.append(bleu_score_4gram.compute_blue_score(sent1, sent2))
                bleu_score.append(bleu_score_avg.compute_blue_score(sent1, sent2))
                b = time.time() - a
                # print(b)
                # sim_score.append(similarity.compute_similarity(sent1, sent2)) # 7*num_sent
            # scores = [bleu_score, bleu1, bleu2, bleu3, bleu4]
                dict = {"sent1":sent1, "sent2":sent2}
                # print(len(dict["sent1"]))
                op1 = pd.DataFrame(dict)
                # print(op1.head())
                op1.to_csv("opnew{}.csv".format(lr), encoding='utf-8')

            
            # print(len(s1[0]), len(s2[0]), len(bleu_score[0]))

   
            scoreavg.append(np.mean(np.array(bleu_score, float), axis=1))
            score1.append(np.mean(np.array(bleu1), axis=1))
            score2.append(np.mean(np.array(bleu2), axis=1))
            score3.append(np.mean(np.array(bleu3), axis=1))
            score4.append(np.mean(np.array(bleu4), axis=1))
       #avg over epochs     
    scoref_avg = np.mean(np.array(scoreavg), axis=0)
    scoref_1 = np.mean(np.array(score1), axis=0)
    scoref_2 = np.mean(np.array(score2), axis=0)
    scoref_3 = np.mean(np.array(score3), axis=0)
    scoref_4 = np.mean(np.array(score4), axis=0)
            
            
            # sim_score = np.array(sim_score)
            # sim_score = np.mean(sim_score, axis=1)
            # score2.append(sim_score)
    
    # score2 = np.mean(np.array(score2), axis=0)

    return {"avg": scoref_avg, "1":scoref_1,"2":scoref_2, "3":scoref_3, "4":scoref_4}#, score2

if __name__ == '__main__':
    args, unk = parser.parse_known_args()

    # args.vocab_file = '/import/antennas/Datasets/hx301/' + args.vocab_file
    args.vocab_file = '' + args.vocab_file
    vocab = json.load(open(args.vocab_file, 'rb'))
    token_to_idx = vocab['token_to_idx']
    idx_to_token = dict(zip(token_to_idx.values(), token_to_idx.keys()))
    num_vocab = len(token_to_idx)
    pad_idx = token_to_idx["<PAD>"]
    start_idx = token_to_idx["<START>"]
    end_idx = token_to_idx["<END>"]


    
    
    """ define optimizer and loss function """
    model1 = Model(args.num_layers_deepsc, num_vocab, num_vocab,
                        num_vocab, num_vocab, args.d_model, args.num_heads,
                        args.dff, 0.1).to(device)

    model2 = Model(args.num_layers_ae, num_vocab, num_vocab,
                        num_vocab, num_vocab, args.d_model, args.num_heads,
                        args.dff, 0.1).to(device)
    
    """Define type and model """
    type = 0
    deepsc = model1
    
    
    pths = ['checkpoints/deepsc-1','checkpoints/deepsc-2']
    bleus_df = pd.DataFrame()
    for p in pths:
        #lr
        print(p)
        
        model_paths = []
        args.checkpoint_path = p
        for fn in os.listdir(args.checkpoint_path):
            
            if not fn.endswith('.pkl'): continue
            idx = int(os.path.splitext(fn)[0].split('_')[-1])  # read the idx of image
            model_paths.append((os.path.join(args.checkpoint_path, fn), idx))

        model_paths.sort(key=lambda x: x[1])  # sort the image by the idx
    
        model_path, _ = model_paths[-1]
        checkpoint = torch.load(model_path)
        deepsc.load_state_dict(checkpoint)
        print('model load!')

        if type == 3:
            ae = AE().to(device)
            bleus_all = performance(args, deepsc, p[-1], type, ae)
        else:
            bleus_all = performance(args, deepsc, p[-1], type)
        # bleu1 = performance(args, SNR, deepsc, "one", ch)
        # bleu2 = performance(args, SNR, deepsc, "two", ch)
        # bleu3 = performance(args, SNR, deepsc, "three", ch)
        # bleu4 = performance(args, SNR, deepsc, "four", ch)
        # bleu_avg = performance(args, SNR, deepsc, "avg", ch)
    #     bleu_dict = {"SNR":SNR, "1gram": bleu1, "2gram":bleu2,"3gram":bleu3,
    #                  "4gram":bleu4,"avg":bleu_avg, "Channel":ch, "LR":p[-1]}
    #     bleus = pd.DataFrame(bleu_dict)
    #     bleus_df.append(bleus, ignore_index=True)
    #     print(bleus.head())
        
    # bleus_df.to_csv("bleus.csv", encoding='utf-8')
        print(p[-1], bleus_all)
    #similarity.compute_similarity(sent1, real)


# len of belu score op = 50 = the no of row items
# but each is a list - but if bleu returns 1 score per sentence, then it should be a float,
# makes no sense to return a score for each word.
